<h2>☁️ Colab-Based Development</h2>

<p>
  This project is built and executed using <strong>Google Colab</strong> to leverage its
  <em>high processing power</em> and access to <strong>free GPU/TPU resources</strong>.
</p>

<p>
  Google Colab provides an ideal environment for running computationally intensive components of this project, including:
</p>

<ul>
  <li>🔠 Embedding large volumes of legal text</li>
  <li>📊 Performing semantic search over vector databases</li>
  <li>🧠 Running inference with the <code>GPT-Neo 1.3B</code> language model</li>
</ul>

<p>
  By using Colab, the project achieves efficient execution without requiring local high-end hardware or paid cloud services.
</p>

In [ ]:
pip install langchain

In [ ]:
pip install -U langchain-community

## Importing Libraries

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers

## function to load file

In [ ]:
def courpus_loader(path):
  loader=DirectoryLoader(path,glob='*.pdf', loader_cls=PyPDFLoader)
  book=loader.load()
  return book

In [ ]:
pip install pypdf

In [ ]:
courpus=courpus_loader("dataset\raw\OJ-L-2016-119-FULL-EN-TXT.pdf")

## function to make chunks of the  courpus

In [ ]:
def chunk_maker(c_size,c_overlap):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=c_size,chunk_overlap=c_overlap)
  final_chunks=text_splitter.split_documents(courpus)
  return final_chunks

In [ ]:
chunks=chunk_maker(c_size=400,c_overlap=40)

In [ ]:
len(chunks)

1868

## function to download embedding model

In [ ]:
def download_embedings(model_name):
  embeddings=HuggingFaceEmbeddings(model_name=model_name)
  return embeddings

In [ ]:
model_embeddings=download_embedings(model_name="sentence-transformers/all-MiniLM-L6-V2")

<ipython-input-9-9c66a0c81023>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model_embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-V2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

## making embeddings of chunks

In [ ]:
hf_embeddings = model_embeddings.embed_documents([chunk.page_content for chunk in chunks])

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS


### alternative method to build vector database 

In [ ]:
vector_store = FAISS(
 embedding_function=embeddings,
 index=index,
 docstore=InMemoryDocstore(),
 index_to_docstore_id={},
 )
vector_store.add_documents(documents=final_chunks)

saving vector database

In [ ]:
vector_store.save_local(folder_path="./Memory",index_name="CWC_index")

In [ ]:
pip install faiss-cpu

In [ ]:
from langchain.vectorstores import FAISS

## making vector database

In [ ]:
vectorstore = FAISS.from_documents(chunks, model_embeddings)

### making prompt using user query

In [ ]:
query = "What is the significance of the Official Journal of the European Union, specifically issue L 119?"
 # Perform similarity search
retrieved_docs = vectorstore.similarity_search(query, k=2)

In [ ]:
page1 = retrieved_docs[1].metadata.get("page", "unknown")
retrieved_context = retrieved_docs[0].page_content + f"\n\n(Page {page1})"
 # Creating the prompt
augmented_prompt=f"""
Given the context below answer the question.
Question: {query}
Context : {retrieved_context}
Remember to answer only based on the context provided and not from any ot
If the question cannot be answered based on the provided context, say I d
s"""

In [ ]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["retrieved_context","query"],
    template=augmented_prompt
)
c_type_k={"prompt":prompt}

In [ ]:
pip install transformers torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 889.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

### downloading model and tokenizer from hugging face

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain.llms import HuggingFacePipeline
# Load LLaMA 2 7B model from Hugging Face
model_name = "EleutherAI/gpt-neo-1.3B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, return_full_text=True)

llm = HuggingFacePipeline(pipeline=pipe)

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-24-7999133c3504>:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


This line sets up a retrieval-based QA system where:

A query is sent by the user

The system retrieves 2 most relevant text chunks from the vectorstore

The language model uses those chunks to generate an informed answer

The original source documents are returned along with the answer

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever(search_kwargs={'k':2}), return_source_documents=True)


### displaying user query and model response

In [26]:
result=qa_chain({"query":query})
print(result['result'])

<ipython-input-26-438d631d6183>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa_chain({"query":query})
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

4.5.2016 L 119/121 Official Jour nal of the European Union EN

4.5.2016 L 119/141 Official Jour nal of the European Union EN

Question: What is the significance of the Official Journal of the European Union, specifically issue L 119?
Helpful Answer:
The L 119 was published in April 2016.
According to the Official Journal of the European Union, issue L 119 is "published within the scope of all the European Union policies as the journal of the European Commission (http://ec.europa.eu/transparency/journals/ec_journal/en/).
The Journal L 119 is also published by the European Commission and its member states."
The Journal L 119, entitled Official Journal of the European Commission, was published on 15.April 2016.

From Wikipedia:

Official Journal of the European Union (OJEU), the official newspaper of the European Union, carries